## Preprocessing

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import librosa
import scipy.io.wavfile
import os
import math
import matplotlib
import scipy.stats

from librosa.display import *
from librosa.effects import *
from keras.utils import to_categorical

import matplotlib.pyplot as plt
%matplotlib inline

/Users/hyunwoogu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
os.chdir('./Samples/')

In [3]:
lists = []
for wave in os.listdir(os.getcwd()):
    if "wav" in wave:
        lists.append(wave)
        
res = []

for wave in lists:
    data, fs = librosa.load(wave)
    data = data/np.ptp(data)
    res.append(data.shape[0])

In [4]:
def onset(data):
    cnt = 0  
    for i in data:
        cnt += 1
        if i > 1e-2:
            return(data[(cnt-1):])
            break
            
def termin(data):
    cnt = 0
    for i in reversed(data):
        cnt += 1
        if i > 1e-2:
            return(data[:(len(data)-cnt+1)])
            break

In [5]:
foo = 'Sad'
bar = 'Happy'

ListSad   = np.array(lists)[[i for i, v in enumerate(lists) if foo in v]]
ListHappy = np.array(lists)[[i for i, v in enumerate(lists) if bar in v]]

Happy_M = np.zeros([129, 410])
Happy_P = np.zeros([129, 410])
Sad_M   = np.zeros([129, 410])
Sad_P   = np.zeros([129, 410])

In [6]:
for wave in ListHappy:
    data, fs = librosa.load(wave)
    trimmed = onset(data)
    trimmed = termin(trimmed)

    trimmed = librosa.effects.time_stretch(trimmed, len(trimmed)/ fs / 1.5)

    STFT    = librosa.stft(trimmed, n_fft=256, hop_length=int(0.01 * 8000), window='hann')
    M, P    = librosa.magphase(STFT)
    logM    = np.log(M)
            
    Happy_M = np.vstack((Happy_M, logM))
    Happy_P = np.vstack((Happy_P, P))    
        
HappyX = np.reshape(Happy_M, [ListHappy.shape[0]+1, 129, 410])
HappyX = HappyX[1:, :, :]
HappyP = np.reshape(Happy_P, [ListHappy.shape[0]+1, 129, 410])
HappyP = HappyP[1:, :, :]

In [8]:
for wave in ListSad:
    data, fs = librosa.load(wave)
    trimmed = onset(data)
    trimmed = termin(trimmed)

    trimmed = librosa.effects.time_stretch(trimmed, len(trimmed)/ fs / 1.5)

    STFT    = librosa.stft(trimmed, n_fft=256, hop_length=int(0.01 * 8000), window='hann')
    M, P    = librosa.magphase(STFT)
    logM    = np.log(M)
            
    Sad_M = np.vstack((Sad_M, logM))
    Sad_P = np.vstack((Sad_P, P))    
        
SadX = np.reshape(Sad_M, [ListSad.shape[0]+1, 129, 410])
SadX = SadX[1:, :, :]
SadP = np.reshape(Sad_P, [ListSad.shape[0]+1, 129, 410])
SadP = SadP[1:, :, :]

In [9]:
dataX = np.vstack((HappyX, SadX))
dataX = np.reshape(dataX, [9, 129, 410])
dataY = np.repeat(np.array([1, 0]), [5, 4])

CheckX = np.vstack((HappyP, SadP))
CheckX = np.reshape(CheckX, [9, 129, 410])
CheckY = np.repeat(np.array([1, 0]), [9, 400])

In [10]:
inx = np.array(range(9))
np.random.shuffle(inx)

dataX = dataX[inx, :, :]
dataY = dataY[inx]

CheckX = CheckX[inx, :, :]
CheckY = CheckY[inx]

dataY = to_categorical(dataY, 2)

## Model

In [13]:
total_epoch = 300
batch_size  = 9
n_hidden = 256
n_input  = 129 * 410
n_noise  = 256
n_class  = 2

In [11]:
tf.reset_default_graph()

In [12]:
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

NameError: name 'n_input' is not defined